In [ ]:
import os
import cv2
from os import listdir
from cv_functions import get_image_dimensions, get_d32, resize_save, split_image_in_4

### Resize Photos to 1280x960 px

Reduce image size for faster uploads.

Change `folder_dir` and `output_dir` to your directories of choice.

In [ ]:
import os
from cv_functions import get_image_dimensions, get_d32, resize_save, split_image_in_4

# for every image in input folder, resize and save in output folder.
folder_dir = "datasets_sorted/TiongBahru"
output_dir = "output/tiongbahru_1280_OG"

print("Resizing...")
for image in os.listdir(folder_dir):
    img_path = folder_dir + f"/{image}"
    h, w = get_image_dimensions(img_path)
    if w > h:
        resize_save(img_path, output_dir, 1280, 960)
        # resize_save(img_path, output_dir, 1600, 1200)
    else:
        resize_save(img_path, output_dir, 960, 1280)
        # resize_save(img_path, output_dir, 1200, 1600)
print("Complete.")

### Increase contrast of image

Increasing the contrast of the images aims to make the visual elements more pronounced, facilitating easier feature recognition by computer algorithms.

<a href='https://stackoverflow.com/questions/39308030/how-do-i-increase-the-contrast-of-an-image-in-python-opencv'>Source code</a>
<br>
<a href='https://www.xrite.com/blog/lab-color-space'>Read about LAB color values</a>
<br>
<br>
Run the cell below.

In [4]:
import cv2

def incr_contrast(image_path, output_folder=None, clip_limit=2, save=True):
    if save:
        img = cv2.imread(image_path)
        new_name = image_path.split('/')[-1].split('.')[0]
    else:
        img = image_path

    # converting to LAB color space
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l_channel, a, b = cv2.split(lab)

    # Applying CLAHE to L-channel
    # feel free to try different values for the limit and grid size:
    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=(8,8))
    cl = clahe.apply(l_channel)

    # merge the CLAHE enhanced L-channel with the a and b channel
    limg = cv2.merge((cl,a,b))

    # Converting image from LAB Color model to BGR color spcae
    enhanced_img = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)

    if not save:
        return enhanced_img
    else:
        if output_folder == None:
            cv2.imwrite(f'output/{new_name}.jpg', enhanced_img)
        else:
            if output_folder[-1] != '/':
                cv2.imwrite(output_folder + f'/{new_name}.jpg', enhanced_img)
            else:
                cv2.imwrite(output_folder + f'{new_name}.jpg', enhanced_img)

#### Comparing original and processed images

Compares original and processed image side by side. <br>
Change `image_path` to the image of your choice.

In [2]:
import cv2
import numpy as np

image_path = 'output/everton_800/IMG_8391_600x800.JPG'

img = cv2.imread(image_path)
# converting to LAB color space
lab= cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
l_channel, a, b = cv2.split(lab)

# Applying CLAHE to L-channel
# feel free to try different values for the limit and grid size:
clahe = cv2.createCLAHE(clipLimit=5)
cl = clahe.apply(l_channel)

# merge the CLAHE enhanced L-channel with the a and b channel
limg = cv2.merge((cl,a,b))

# Converting image from LAB Color model to BGR color spcae
enhanced_img = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)

new_img = cv2.resize(img, (800, 600))
new_enhanced_img = cv2.resize(enhanced_img, (800, 600))

# Stacking the original image with the enhanced image
result = np.hstack((new_img, new_enhanced_img))
cv2.imshow('Result', result)
cv2.waitKey(0)

-1

#### Increase contrast of one image

Saves processed image in `output_dir`. <br>
Change `img_path` and `output_dir` to the image and save location of your choice.

In [ ]:
import os
import datetime
now = datetime.datetime.now()

# for every image in input folder, increase contrast and save in output folder.
img_path = "datasets_sorted/PunggolMeridian_MRT/IMG_0019.JPG"
output_dir = "output/test"

print("Increasing contrast...")
incr_contrast(image_path=img_path, output_folder=output_dir, clip_limit=5)
    
print("Complete.")

#### Increase contrast of all images in folder

Saves all processed images from a folder into `output_dir`. <br>
Change `folder_dir` and `output_dir` to the image and save location of your choice.

<br>May not run fully if folder contains format other than images.

In [1]:
import os
import datetime

now = datetime.datetime.now()

# for every image in input folder, increase contrast and save in output folder.
folder_dir = "output/punggol_blk160_1280"
output_dir = "datasets_processed/punggol_blk160_1280"

print("Increasing contrast...")
for image in os.listdir(folder_dir):
    incr_contrast(image_path=folder_dir + f"/{image}", output_folder=output_dir, clip_limit=5)
    
print("Complete.")

Increasing contrast...
Complete.


### Video processing

#### Extract frames from video

The following functions assist in extracting frames from video and preprocesses them.

In [23]:
import cv2

def preprocess_frames(video_path, output_folder, frame_count=-1):
    vidObj = cv2.VideoCapture(video_path)
    count = 0
    success = 1
    while success:
        success, image = vidObj.read()
        try:
            # preprocess
            # image = incr_contrast(image_path=image, clip_limit=5, save=False)
            sized_img = cv2.resize(image, (800, 800))

            # save in output folder
            cv2.imwrite(f"{output_folder}/frame{count_to_string(count)}.jpg", sized_img)
            count += 1
        except Exception as e:
            print(e)
            success = 1
            count += 1

        if frame_count != -1 and count >= frame_count:
            break

def incr_contrast(image_path, output_folder=None, clip_limit=2, save=True):
    if save:
        img = cv2.imread(image_path)
        new_name = image_path.split('/')[-1].split('.')[0]
    else:
        img = image_path

    # converting to LAB color space
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l_channel, a, b = cv2.split(lab)

    # Applying CLAHE to L-channel
    # feel free to try different values for the limit and grid size:
    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=(8,8))
    cl = clahe.apply(l_channel)

    # merge the CLAHE enhanced L-channel with the a and b channel
    limg = cv2.merge((cl,a,b))

    # Converting image from LAB Color model to BGR color spcae
    enhanced_img = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)

    if not save:
        return enhanced_img
    else:
        if output_folder == None:
            cv2.imwrite(f'output/{new_name}.jpg', enhanced_img)
        else:
            if output_folder[-1] != '/':
                cv2.imwrite(output_folder + f'/{new_name}.jpg', enhanced_img)
            else:
                cv2.imwrite(output_folder + f'{new_name}.jpg', enhanced_img)

def count_to_string(count):
    l = 8
    return "0"*(l-len(str(count))) + str(count)

#### Run the following to extract frames from video

Change `video_path` and `output_folder` to source video and save location of your choice. <br>
`frame_count` extracts the first x frames of the video. If extract all, set as -1.

In [24]:
preprocess_frames(video_path="datasets_project/Punggol_EdgeField BLK111/GH012496.MP4", output_folder="output/videos/GH012496", frame_count=300)

OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4062: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4062: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4062: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4062: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4062: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4062: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'



#### Make images into video

Change `source_folder` and `save_path` to directories of choice. <br> <br>
May not write fully if folder contains different sized images/format.

In [25]:
import cv2
import os

source_folder = "output/videos/GH012496"
save_path = "output/videos/GH012496_trimmed.mp4"

video = cv2.VideoWriter(save_path, 0, 30, (800, 800)) 

for image in os.listdir(source_folder):
    print(image)
    video.write(cv2.imread(source_folder+f"/{image}"))
video.release()

frame00000000.jpg
frame00000001.jpg
frame00000002.jpg
frame00000003.jpg
frame00000004.jpg
frame00000005.jpg
frame00000006.jpg
frame00000007.jpg
frame00000008.jpg
frame00000009.jpg
frame00000010.jpg
frame00000011.jpg
frame00000012.jpg
frame00000013.jpg
frame00000014.jpg
frame00000015.jpg
frame00000016.jpg
frame00000017.jpg
frame00000018.jpg
frame00000019.jpg
frame00000026.jpg
frame00000027.jpg
frame00000028.jpg
frame00000029.jpg
frame00000030.jpg
frame00000031.jpg
frame00000032.jpg
frame00000033.jpg
frame00000034.jpg
frame00000035.jpg
frame00000036.jpg
frame00000037.jpg
frame00000038.jpg
frame00000039.jpg
frame00000040.jpg
frame00000041.jpg
frame00000042.jpg
frame00000043.jpg
frame00000044.jpg
frame00000045.jpg
frame00000046.jpg
frame00000047.jpg
frame00000048.jpg
frame00000049.jpg
frame00000050.jpg
frame00000051.jpg
frame00000052.jpg
frame00000053.jpg
frame00000054.jpg
frame00000055.jpg
frame00000056.jpg
frame00000057.jpg
frame00000058.jpg
frame00000059.jpg
frame00000060.jpg
frame00000